# Programming Assessment for the Certificate in Machine Learning

In [1]:
import pandas as pd
from scipy.optimize import curve_fit

## Load Data

In [2]:
df = pd.read_csv('inspections.csv', parse_dates=['activity_date'])

In [3]:
print (df.shape)
df.head(5)

(191371, 20)


,activity_date,employee_id,facility_address,facility_city,facility_id,facility_name,facility_state,facility_zip,grade,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id,score,serial_number,service_code,service_description
0,2017-05-09,EE0000593,17660 CHATSWORTH ST,GRANADA HILLS,FA0175397,HOVIK'S FAMOUS MEAT & DELI,CA,91344,A,OW0181955,JOHN'S FAMOUS MEAT & DELI INC.,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,HOVIK'S FAMOUS MEAT & DELI,ACTIVE,PR0168541,98,DAHDRUQZO,1,ROUTINE INSPECTION
1,2017-04-10,EE0000126,3615 PACIFIC COAST HWY,TORRANCE,FA0242138,SHAKEY'S PIZZA,CA,90505,A,OW0237843,"SCO, LLC",RESTAURANT (61-150) SEATS HIGH RISK,1638,SHAKEY'S PIZZA,ACTIVE,PR0190290,94,DAL3SBUE0,1,ROUTINE INSPECTION
2,2017-04-04,EE0000593,17515 CHATSWORTH ST,GRANADA HILLS,FA0007801,BAITH AL HALAL,CA,91344,A,OW0031150,SABIR MOHAMMAD SHAHID,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,BAITH AL HALAL,INACTIVE,PR0036723,95,DAL2PIKJU,1,ROUTINE INSPECTION
3,2017-08-15,EE0000971,44455 VALLEY CENTRAL WAY,LANCASTER,FA0013858,FOOD 4 LESS #306,CA,93536,A,OW0012108,"FOOD 4 LESS, INC.",RESTAURANT (0-30) SEATS HIGH RISK,1632,FOOD 4 LESS DELI/BAKERY#306,ACTIVE,PR0039905,98,DA0ZMAJXZ,1,ROUTINE INSPECTION
4,2016-09-26,EE0000145,11700 SOUTH ST,ARTESIA,FA0179671,PHO LITTLE SAIGON,CA,90701,A,OW0185167,PHO SOUTH ST INC,RESTAURANT (61-150) SEATS HIGH RISK,1638,PHO LITTLE SAIGON,ACTIVE,PR0173311,96,DA41DBXA2,1,ROUTINE INSPECTION


In [4]:
df.dtypes

activity_date          datetime64[ns]
employee_id                    object
facility_address               object
facility_city                  object
facility_id                    object
facility_name                  object
facility_state                 object
facility_zip                   object
grade                          object
owner_id                       object
owner_name                     object
pe_description                 object
program_element_pe              int64
program_name                   object
program_status                 object
record_id                      object
score                           int64
serial_number                  object
service_code                    int64
service_description            object
dtype: object

## Assessment Questions

### Employees

In [5]:
employee_scores = df.groupby(['employee_id'])['score'].agg('mean').reset_index()

In [6]:
top3_employee_scores = employee_scores.sort_values('score', ascending=[0]).reset_index(drop=True).iloc[:3]
bottom3_employee_scores = employee_scores.sort_values('score', ascending=[1]).reset_index(drop=True).iloc[:3]

In [7]:
print ("Employees with top 3 mean score:   ", list(top3_employee_scores['employee_id']))
print ("Employees with bottom 3 mean score:", list(bottom3_employee_scores['employee_id']))

Employees with top 3 mean score:    ['EE0000252', 'EE0000284', 'EE0000259']
Employees with bottom 3 mean score: ['EE0000821', 'EE0000699', 'EE0000719']


### Facilities

In [8]:
facility_scores = df.groupby(['facility_city'])['score'].agg('mean').reset_index().sort_values('score', ascending=[1]).reset_index(drop=True).iloc[[0,-1]]
facility_locations = df.groupby(['facility_name'])['facility_city'].nunique().reset_index().sort_values('facility_city', ascending=[0])

In [9]:
print ("City with highest mean score:", facility_scores['facility_city'].iloc[-1])
print ("City with lowest mean score: ", facility_scores['facility_city'].iloc[0])
print ("Facility with the most locations:", facility_locations['facility_name'].iloc[0])

City with highest mean score: SOUTH SAN GABRIEL
City with lowest mean score:  LEONA VALLEY
Facility with the most locations: SUBWAY


### Annual Scores

In [10]:
df['activity_date'].dt.year.unique()

array([2017, 2016, 2015], dtype=int64)

In [11]:
city_year_score = df.groupby(['facility_city', df['activity_date'].dt.year])['score'].agg('mean').reset_index()

In [12]:
city_by_year = city_year_score.pivot(index='facility_city', columns='activity_date', values='score').reset_index()
city_by_year = city_by_year.T.fillna(city_by_year.mean(axis=1)).T

In [13]:
def f(x, m, b):
    return m*x + b

def get_slope(r_):
    x_ = [2015, 2016, 2017]
    y_ = [r_[2015], r_[2016], r_[2017]]

    m,b = curve_fit(f, x_, y_, [1, 2000])[0]
    
    return m

In [14]:
city_by_year.loc[:, 'score_change_by_year'] = city_by_year.apply(lambda r: get_slope(r), axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [15]:
city_by_year_biggest_change = city_by_year.sort_values('score_change_by_year', ascending=[1]).reset_index(drop=True)

In [16]:
print ("City improving the most:", city_by_year_biggest_change['facility_city'].iloc[-1])
print ("City deteriorating the most: ", city_by_year_biggest_change['facility_city'].iloc[0])

City improving the most: PEARBLOSSOM
City deteriorating the most:  LAKE HUGHES


### Initiated by Owner

In [17]:
df['service_description'].value_counts()

ROUTINE INSPECTION                  188463
OWNER INITIATED ROUTINE INSPECT.      2908
Name: service_description, dtype: int64

In [18]:
facility_by_service_score = df.groupby(['facility_id', 'service_description'])['score'].agg('mean').reset_index()
facility_by_service = facility_by_service_score.pivot(index='facility_id', columns='service_description', values='score').reset_index()

In [19]:
cond_both = (~facility_by_service['OWNER INITIATED ROUTINE INSPECT.'].isnull()) & (~facility_by_service['ROUTINE INSPECTION'].isnull())
facility_both_services = facility_by_service[cond_both].reset_index(drop=True)
cond_higher = (facility_both_services['OWNER INITIATED ROUTINE INSPECT.']>facility_both_services['ROUTINE INSPECTION'])

In [20]:
percentage_owner_higher_score = cond_higher.sum() * 100./facility_both_services.shape[0]
print ("Percentage of higher establishments with higher score when owner initiated: %.1f%%" % (percentage_owner_higher_score))

Percentage of higher establishments with higher score when owner initiated: 83.0%
